In [37]:
%%writefile cross_zero_bot.py
import time
import telebot
from telebot import types
bot = telebot.TeleBot(token)
fplayer = 0
splayer = 0
fstate = ""
sstate = ""
turn = -1
state = ""
tme = 0
ids = set()
invoker = 0
ws = False
logger = open('log.txt','w')
strwinners = ['111000000', '000111000', '000000111','100010001', '001010100', '100100100', '010010010', '001001001']
def wstate(state):
    global strwinners
    global logger
    locstate = ''
    for w in strwinners:
        for i,sym in enumerate(w):
            locstate += str((int(sym) and int(state[i])))
        if locstate in strwinners:
            return True
        locstate = ''
    return False

@bot.message_handler(func = lambda x : x.text == 'notify')
def broadcast(message):
    global logger
    global ids
    logger.write(str(message) + '\n')
    if message.from_user.username != 'eles13':
        return
    try:
        with open('chatids.txt', 'r') as fin:
            lines = fin.readlines()
        for line in lines:
            ids.add(line[:-1])
        with open('chatids.txt', 'w') as fout:
            for idd in ids:
                fout.write(idd + '\n')
        bot.send_message(message.chat.id, 'Ready for input')
        bot.register_next_step_handler(message, broadcasting_itself)
        return
    except Exxception as e:
        logger.write('\n\n\nException in broadcasting: \n' + str(e) + '\n\n\n')

def broadcasting_itself(message):
    global ids
    for idd in ids:
        bot.send_message(int(idd), message.text)
        
@bot.message_handler(commands=['start'])
def send_welcome(message):
    global logger
    global turn
    turn = 0
    with open('chatids.txt', 'a') as storage:
        storage.write(str(message.chat.id) + '\n')
    bot.reply_to(message, "Hello, I'm going to provide you with cross-zero game")
    logger.write('<<Command Start in' + str(message) + '>>\n')
    return

@bot.message_handler(commands=['game'])
def start_game(message):
    global tme
    global turn 
    global state
    global fstate
    global sstate
    global fplayer
    global splayer
    global logger
    global ws
    try:
        if turn == -1:
            bot.send_message(message.chat.id, 'First use "start" command, please')
        turn = 0
        state = '*********'
        fstate = '000000000'
        sstate = '000000000'
        fplayer = 0
        splayer = 0
        bot.send_message(message.chat.id, 'Lets start a game, the first two of you who reply with "I" will play')
        tme =time.time()
        bot.register_next_step_handler(message, getfirstplayer)
    except Exeption as e:
        logger.write('\n\n\n<<Exception Game in ' + str(message) + '>>\n' + str(e) + '\n\n\n')   
    return

def getfirstplayer(message):
    global tme
    global fplayer
    global logger
    try:
        if message.text.lower() == 'i':
            fplayer = message.from_user.id
            bot.reply_to(message, "Ok, " + message.from_user.first_name + ", you are the first")
            tme = time.time()
            logger.write('<<Registered First player ' + str(message) + '>>\n')
            bot.register_next_step_handler(message, getsecondplayer)
        elif message.text.lower() != 'stop' and time.time() - tme < 15:
            bot.reply_to(message, 'Wrong, waiting for "I"')
            bot.register_next_step_handler(message, getfirstplayer)
        elif message.text.lower() != 'stop':
            bot.send_message(message.chat.id, "Timeout, bros, try again")
        else:
            bot.send_message(message.chat.id, 'Ok ok, stoppping')
    except Exception as e:
        logger.write('\n\n\n<<Exception First Player in ' + str(message) + '>>\n' + str(e) + '\n\n\n') 
        
        
def getsecondplayer(message):
    global tme
    global fplayer
    global splayer
    global logger
    try:
        if message.text.lower() == 'i' and message.from_user.id != fplayer:
            splayer = message.from_user.id
            bot.reply_to(message, "Ok, " + message.from_user.first_name + ", you are the second")
            tme = 0
            logger.write('<<Registered First player ' + str(message) + '>>\n')
            game_itself(message.chat.id)
        elif message.from_user.id == fplayer and message.text.lower() != 'stop' and time.time() - tme < 15:
            bot.reply_to(message, 'No no, another one')
            bot.register_next_step_handler(message, getsecondplayer)
        elif message.text.lower() != 'stop' and time.time() - tme < 15:
            bot.reply_to(message, 'Wrong, waiting for "I"')
            bot.register_next_step_handler(message, getsecondplayer)
        elif message.text.lower() != 'stop':
            bot.send_message(message.chat.id, "Timeout, bros, try again")
        else:
            bot.send_message(message.chat.id, 'Ok ok, stoppping')
    except Exception as e:
        logger.write('\n\n\n<<Exception Second Player in ' + str(message) + '>>\n' + str(e) + '\n\n\n') 
        
def game_itself(chatid, msgid = None):
    global state
    global fplayer
    global splayer
    global logger
    global ws
    try:
        whose = fplayer
        if turn%2 == 1:
            whose = splayer
        keyboard = types.InlineKeyboardMarkup()
        key0 = types.InlineKeyboardButton(text=state[0], callback_data='0')
        key1 = types.InlineKeyboardButton(text=state[1], callback_data='1')
        key2 = types.InlineKeyboardButton(text=state[2], callback_data='2')
        keyboard.row(key0, key1, key2)
        key3 = types.InlineKeyboardButton(text=state[3], callback_data='3')
        key4 = types.InlineKeyboardButton(text=state[4], callback_data='4')
        key5 = types.InlineKeyboardButton(text=state[5], callback_data='5')
        keyboard.row(key3, key4, key5)
        key6 = types.InlineKeyboardButton(text=state[6], callback_data='6')
        key7 = types.InlineKeyboardButton(text=state[7], callback_data='7')
        key8 = types.InlineKeyboardButton(text=state[8], callback_data='8')
        keyboard.row(key6, key7, key8)
        UsrInfo = bot.get_chat_member(whose, whose).user
        msg = UsrInfo.first_name + ', your turn'
        if state == '*********':
            ws = False
            bot.send_message(chatid, text = msg, reply_markup=keyboard)
        else: 
            bot.edit_message_text(chat_id = chatid, message_id = msgid, text = msg, reply_markup=keyboard)
        logger.write('<<Successful keyboard update on chat ' + str(chatid) + '>>\n')
    except Exception as e:
        logger.write('\n\n\n<<Exception keyboard update in ' + str(chatid) + ' msg ' + str(msgid) + '>>\n' + str(e) + '\n\n\n')
        
@bot.callback_query_handler(func=lambda call: True)
def changeState(call):
    global turn 
    global state
    global fstate
    global sstate
    global fplayer
    global splayer
    global logger
    global ws
    try:
        if ws:
            return
        sym = 'X'
        whose = fplayer
        curstate = fstate
        teststate = ''
        if turn%2 == 1:
            sym = 'O'
            whose = splayer
        if call.from_user.id != whose:
            bot.reply_to(call.message, call.from_user.first_name + ', do not interrupt the game, please')
            game_itself(call.message.chat.id)
        if state[int(call.data)] != '*':
            bot.send_message(call.message.chat.id, 'Wrong button')
            game_itself(call.message.chat.id)
        state = list(state)
        state[int(call.data)] = sym
        state = ''.join(state)
        if turn%2 == 0:
            fstate = list(fstate)
            fstate[int(call.data)] = '1'
            fstate = ''.join(fstate)
            teststate = fstate
        else:
            sstate = list(sstate)
            sstate[int(call.data)] = '1'
            sstate = ''.join(sstate)
            teststate = sstate
        
        if not wstate(teststate):
            if turn == 7:
                ws = True
                bot.send_message(call.message.chat.id, "It's a draw!")
                logger.write('<<Draw in ' + str(call) + '>>\n')
                return
            turn += 1
            logger.write('<<Successfully changed state on turn ' + str(turn) + ' in '+ str(call) + '>>\n')
            game_itself(call.message.chat.id, call.message.message_id)
        else:
            UsrInfo = bot.get_chat_member(whose, whose).user
            bot.send_message(call.message.chat.id, 'Congratulations! ' + UsrInfo.first_name + ' wins!')
            ws = True
            logger.write('<<Win in ' + str(call) + '>>\n')
    except Exception as e:
        logger.write('\n\n\n<<Exception changing state in ' + str(call) + '>>\n' + str(e) + '\n\n\n')
        
        
bot.polling(none_stop=True, interval=0)

Overwriting cross_zero_bot.py
